# Flow Counter

* This notebook is used to count flows, in respect to each geochem group, from different wells.
-----------------------------------------------------------------------------------------------------------------   
* Main parts include:
     * A flow counter algorithm
     * A function generalizing the flow-counter algorithm -- TODO

## Importing libraries and data

In [15]:
# Importing libraries
import pandas as pd

# Reading .csv
# data = pd.read_csv(r'Data/CSV/1-AV-001-PR.csv', sep=';') ## Dataset1
data = pd.read_csv(r'Data/CSV/1-CS-002-PR.csv', sep=';') ##Dataset2
data.head()

,Well,Unit,Facies,Facies Thickness,Flow thickness,Core Proportion
0,1-CS-002-PR,Paranapanema,Simple lava (basic | massive interior),"3,65",NaN,NaN
1,1-CS-002-PR,Paranapanema,Simple lava (basic | L. crust),"1,12",NaN,NaN
2,1-CS-002-PR,Paranapanema,Siliciclastics,"1,52",NaN,NaN
3,1-CS-002-PR,Paranapanema,Simple lava (basic | U. crust),"11,6","37,28","0,650214592"
4,1-CS-002-PR,Paranapanema,Simple lava (basic | massive interior),"24,24",NaN,NaN


## Flow Counter Algorithm
* This code is used to run throught the WellCad data and counting the # flows per geochemical group.
-----------------------------------------------------------------------------------------------------------------   
* Things considered to indicate flow "change":
    * Upper crust of lava flow
    * Compound flows are counted as singular flow
    * Interbeds indicate flow changes
    * Unit changes indicate flow changes
-----------------------------------------------------------------------------------------------------------------    
* Output:
    * A flow_list list which contains the exact same number of values as the input Pandas DataFrame. 
    * This list can then be appended to the given dataframe to generate a flow# column.

In [13]:
# Flow Counter v2 -- Accounting for Unit Intercalation
# PROBLEMS -- Is not outputting the correct dictionary I want (unit:flow #)
# Problem 2 -- Flow counter is not resseting between Pitanga/Paranapanema (check this). Related to #2 dataset

# Setting up variables
flow_counter = 0
flow_list = []
flow_dict = {}

# Algorithm
for i in range(len(data)):
    if i==0: # Setting up the first iteration
        flow_counter += 1
        flow_dict[data['Facies'].iloc[i]] = flow_counter
        flow_list.append(flow_counter)
    elif i == (len(data)-1): # Checking for the end of the list (working around out-of-bounds bug)
        flow_list.append(flow_counter)
        break
    elif data['Facies'].iloc[i] == "Siliciclastics" or data['Facies'].iloc[i] == "Volcaniclastics (undiff.)": # Check for siliciclastic OR volcaniclastic; append a "-" for these units:
        flow_list.append('-')        
    else: # If we're indeed dealing with lava flow data  
        if data['Unit'].iloc[i] == data['Unit'].iloc[i-1]: # If unit is the same as before
            if data['Facies'].iloc[i] == 'Simple lava (basic | L. crust)' or data['Facies'].iloc[i] == 'Compound lava (basic)': # Check for Simple lower crust OR compound flows:
                flow_list.append(flow_counter)
                flow_counter += 1
            else:
                flow_list.append(flow_counter)
        else: # If Unit changes
            if data['Unit'].iloc[i] in flow_dict: # If unit has already been added as a key to the flow_dict
                flow_dict[data['Unit'].iloc[i-1]] = flow_counter
                flow_counter = flow_dict[data['Unit'].iloc[i]]
                flow_list.append(flow_counter)
            else: # If unit is NOT already associated with a value in the flow_dict
                flow_dict[data['Unit'].iloc[i-1]] = flow_counter
                flow_counter = 1
                flow_dict[data['Unit'].iloc[i]] = flow_counter
                flow_list.append(flow_counter)
                if data['Facies'].iloc[i]  == 'Compound lava (basic)': # Working around the situtation where the unit changes into a "compound lava" facies.
                    flow_counter += 1
                else:
                    continue

# Checking  
print(flow_dict)
print(flow_list)
data['Flow#'] = flow_list
data.head(20)

{'Simple lava (basic | U. crust)': 1, 'Paranapanema': 33, 'Pitanga': 34, 'Vale do Sol': 1}
[1, 1, 1, '-', 2, 2, 2, '-', 3, 3, 3, '-', 4, 4, 4, 5, '-', 6, 7, 7, 7, 8, 9, 9, 9, '-', 10, 10, 10, '-', 11, 11, 11, '-', 12, 12, 12, '-', 13, 13, 13, 14, 14, 14, '-', 15, 16, 16, 16, 17, 18, 18, 18, '-', 19, 19, 19, '-', 20, 20, 20, 21, '-', 22, 22, 22, '-', 23, 24, 24, 24, '-', 25, 25, 25, 26, 26, 26, '-', 27, 27, 27, 1, 1, 1, 2, 3, 3, 3, 4, 28, 28, 28, '-', 29, 30, 30, 30, 31, 32, 32, 32, '-', 5, '-', 5, 6, 6, 6, '-', 7, 8, 8, 8, 9, 10, 11, 11, 11, '-', 12, 12, 12, '-', 13, 13, 13, 14, 14, 14, '-', 15, 16, 16, 16, 17, 17, 17, 18, '-', 19, 19, 19, 20, 20, 20, 21, 21, 21, '-', 22, 22, 22, 23, 24, 24, 24, '-', 25, 25, 25, 26, 26, 26, '-', 27, 28, 28, 28, 29, 29, 29, '-', 30, 30, 30, 31, 32, 32, 32, '-', 33, 33, 33, 1, 2, 2, 2, 3, 4, 4, 4, 5]


,Well,Unit,Facies,Facies Thickness,Flow thickness,Core Proportion,Flow#
0,1-AV-001-PR,Paranapanema,Simple lava (basic | U. crust),"11,2","31,36","0,589285714",1
1,1-AV-001-PR,Paranapanema,Simple lava (basic | massive interior),"18,48",NaN,NaN,1
2,1-AV-001-PR,Paranapanema,Simple lava (basic | L. crust),"1,68",NaN,NaN,1
3,1-AV-001-PR,Paranapanema,Siliciclastics,"1,23","1,23",NaN,-
4,1-AV-001-PR,Paranapanema,Simple lava (basic | U. crust),"29,81","88,48","0,643196203",2
5,1-AV-001-PR,Paranapanema,Simple lava (basic | massive interior),"56,91",NaN,NaN,2
6,1-AV-001-PR,Paranapanema,Simple lava (basic | L. crust),"1,76",NaN,NaN,2
7,1-AV-001-PR,Paranapanema,Siliciclastics,"2,16","2,16",NaN,-
8,1-AV-001-PR,Paranapanema,Simple lava (basic | U. crust),"4,48","13,36","0,616766467",3
9,1-AV-001-PR,Paranapanema,Simple lava (basic | massive interior),"8,24",NaN,NaN,3


## Putting the algorithm inside a function for better functionality

## Others /// Notes: